In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import re
import preprocessor as p
import emoji
import codecs
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords = set(stopwords.words('english')) - set(['no', 'not'])

import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
tf.get_logger().setLevel('ERROR')

In [2]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
}

In [3]:
def write_files(dataframe):
    print("Writing files...")
    for i in tqdm(range(len(dataframe.loc[:, "Text"].values))):
        if i < len(dataframe.loc[:, "Text"].values)*0.7:
            if dataframe.loc[i, "Sentiment"] == 1:
                file_name = str(i) + ".txt"
                with open('sentiment/train/pos/'+file_name, 'w', encoding='utf-8') as f:
                    f.write(dataframe.loc[i, "Text"])
            elif dataframe.loc[i, "Sentiment"] == -1:
                file_name = str(i) + ".txt"
                with open('sentiment/train/neg/'+file_name, 'w', encoding='utf-8') as f:
                    f.write(dataframe.loc[i, "Text"])
            else:
                file_name = str(i) + ".txt"
                with open('sentiment/train/neu/'+file_name, 'w', encoding='utf-8') as f:
                    f.write(dataframe.loc[i, "Text"])
        else:
            if dataframe.loc[i, "Sentiment"] == 1:
                file_name = str(i) + ".txt"
                with open('sentiment/test/pos/'+file_name, 'w', encoding='utf-8') as f:
                    f.write(dataframe.loc[i, "Text"])
            elif dataframe.loc[i, "Sentiment"] == -1:
                file_name = str(i) + ".txt"
                with open('sentiment/test/neg/'+file_name, 'w', encoding='utf-8') as f:
                    f.write(dataframe.loc[i, "Text"])
            else:
                file_name = str(i) + ".txt"
                with open('sentiment/test/neu/'+file_name, 'w', encoding='utf-8') as f:
                    f.write(dataframe.loc[i, "Text"])

In [4]:
def train_bert_model():
    AUTOTUNE = tf.data.AUTOTUNE
    batch_size = 32
    seed = 42

    raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
        'sentiment/train',
        batch_size=batch_size,
        validation_split=0.2,
        subset='training',
        seed=seed)

    class_names = raw_train_ds.class_names
    train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

    val_ds = tf.keras.preprocessing.text_dataset_from_directory(
        'sentiment/train',
        batch_size=batch_size,
        validation_split=0.2,
        subset='validation',
        seed=seed)

    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

    test_ds = tf.keras.preprocessing.text_dataset_from_directory(
        'sentiment/test',
        batch_size=batch_size)

    test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

    bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
    tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
    tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

    bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
    bert_model = hub.KerasLayer(tfhub_handle_encoder)

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(net)
    classifier_model = tf.keras.Model(text_input, net)

    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = tf.metrics.Accuracy()
    epochs = 1
    steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)
    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                              num_train_steps=num_train_steps,
                                              num_warmup_steps=num_warmup_steps,
                                              optimizer_type='adamw')

    classifier_model.compile(optimizer='rmsprop', 
                             loss='sparse_categorical_crossentropy', 
                             metrics=['accuracy'])

    print(f'Training model with {tfhub_handle_encoder}')
    history = classifier_model.fit(x=train_ds, validation_data=val_ds, epochs=epochs)

    loss, accuracy = classifier_model.evaluate(test_ds)
    return loss, accuracy

In [5]:
dataset = pd.read_csv('sentiment.csv')

In [6]:
write_files(dataset)

  2%|▏         | 270/11970 [00:00<00:04, 2520.85it/s]

Writing files...


100%|██████████| 11970/11970 [00:09<00:00, 1304.08it/s]


In [7]:
loss, accuracy = train_bert_model()

Found 8379 files belonging to 3 classes.
Using 6704 files for training.
Found 8379 files belonging to 3 classes.
Using 1675 files for validation.
Found 3591 files belonging to 3 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
113/113 [==============================] - 70s 619ms/step - loss: 1.1740 - accuracy: 0.3386


In [8]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.lower()
    dataset.loc[i, "Text"] = v

In [9]:
write_files(dataset)

  2%|▏         | 246/11970 [00:00<00:07, 1608.74it/s]

Writing files...


100%|██████████| 11970/11970 [00:03<00:00, 3753.59it/s]


In [10]:
loss, accuracy = train_bert_model()

Found 8379 files belonging to 3 classes.
Using 6704 files for training.
Found 8379 files belonging to 3 classes.
Using 1675 files for validation.
Found 3591 files belonging to 3 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
113/113 [==============================] - 70s 623ms/step - loss: 1.1729 - accuracy: 0.3386


In [11]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.replace('#', ' ')
    v = v.replace('@', ' ')
    v = p.clean(v)
    dataset.loc[i, "Text"] = v

In [12]:
write_files(dataset)

  2%|▏         | 237/11970 [00:00<00:05, 2307.53it/s]

Writing files...


100%|██████████| 11970/11970 [00:05<00:00, 2313.46it/s]


In [13]:
loss, accuracy = train_bert_model()

Found 8379 files belonging to 3 classes.
Using 6704 files for training.
Found 8379 files belonging to 3 classes.
Using 1675 files for validation.
Found 3591 files belonging to 3 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
113/113 [==============================] - 72s 633ms/step - loss: 1.1514 - accuracy: 0.3386


In [14]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.replace("’", "'")
    for key in contractions:
        if key in v:
            v = re.sub(key, contractions[key], v)
    dataset.loc[i, "Text"] = v

In [15]:
write_files(dataset)

  2%|▏         | 260/11970 [00:00<00:04, 2598.03it/s]

Writing files...


100%|██████████| 11970/11970 [00:09<00:00, 1294.69it/s]


In [16]:
loss, accuracy = train_bert_model()

Found 8379 files belonging to 3 classes.
Using 6704 files for training.
Found 8379 files belonging to 3 classes.
Using 1675 files for validation.
Found 3591 files belonging to 3 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
113/113 [==============================] - 73s 650ms/step - loss: 1.1737 - accuracy: 0.3386


In [17]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = codecs.decode(v, 'unicode_escape')
    v = emoji.demojize(v.encode('utf-16', 'surrogatepass').decode('utf-16'))
    dataset.loc[i, "Text"] = v

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: invalid escape sequence '\ '
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
write_files(dataset)

  2%|▏         | 296/11970 [00:00<00:03, 2954.85it/s]

Writing files...


100%|██████████| 11970/11970 [00:09<00:00, 1263.24it/s]


In [19]:
loss, accuracy = train_bert_model()

Found 8379 files belonging to 3 classes.
Using 6704 files for training.
Found 8379 files belonging to 3 classes.
Using 1675 files for validation.
Found 3591 files belonging to 3 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
113/113 [==============================] - 70s 620ms/step - loss: 1.1294 - accuracy: 0.3386


In [20]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = re.sub("[^a-z0-9 ]", " ", v).strip()
    v = re.sub(" +", " ", v)
    dataset.loc[i, "Text"] = v

In [21]:
write_files(dataset)

  2%|▏         | 258/11970 [00:00<00:04, 2544.74it/s]

Writing files...


100%|██████████| 11970/11970 [00:10<00:00, 1165.29it/s]


In [22]:
loss, accuracy = train_bert_model()

Found 8379 files belonging to 3 classes.
Using 6704 files for training.
Found 8379 files belonging to 3 classes.
Using 1675 files for validation.
Found 3591 files belonging to 3 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
113/113 [==============================] - 70s 619ms/step - loss: 1.1356 - accuracy: 0.3386


In [23]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    temp_v = v.split()
    l = WordNetLemmatizer()
    v = [l.lemmatize(word) for word in temp_v if not word in stopwords]
    v = ' '.join(v)
    dataset.loc[i, "Text"] = v

In [24]:
write_files(dataset)

  3%|▎         | 300/11970 [00:00<00:03, 2998.11it/s]

Writing files...


100%|██████████| 11970/11970 [00:10<00:00, 1161.07it/s]


In [25]:
loss, accuracy = train_bert_model()

Found 8379 files belonging to 3 classes.
Using 6704 files for training.
Found 8379 files belonging to 3 classes.
Using 1675 files for validation.
Found 3591 files belonging to 3 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
113/113 [==============================] - 71s 625ms/step - loss: 1.1155 - accuracy: 0.3386
